In [ ]:
!pip install numpy
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install catboost

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', 200)

In [3]:
X_train_dataset = pd.read_csv("X.csv")
y_train_dataset = pd.read_csv("y.csv")

In [4]:
df = pd.merge(X_train_dataset, y_train_dataset, on=["engine_id", "flight_datetime", "flight_phase"])

In [6]:
def get_score_by_parameter(predict_field):
    df = pd.merge(X_train_dataset, y_train_dataset, on=["engine_id", "flight_datetime", "flight_phase"])
    df = df[df[predict_field].notna()]
    df = df.fillna(-100)
    X = df.drop(columns=[predict_field])
    X = X.drop(columns=[
        "engine_id", "aircraft_id", "flight_datetime",
        "flight_phase", "engine_family", "engine_type", "manufacturer",
        "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"])
    y = df[predict_field]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=100, verbose=False)
    model.fit(X_train, y_train)
    catboost_predicted = model.predict(X_test)
    catboost_score = mean_squared_error(y_test, catboost_predicted, squared=False)
    catboost_predicted = model.predict(X)
    return catboost_score

In [7]:
scores = pd.DataFrame(columns=["parameter", "rmse", ".0", ".1", ".5", ".9", "1"])
output_parameters = y_train_dataset.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])

for param in tqdm(output_parameters):
    try:
        score = get_score_by_parameter(param)
        qua = df[param].quantile([.0, .1, .5, .9, 1])
        qua = list(qua)
        scores.loc[len(scores)] = [param, score] + qua
    except:
        continue

100%|███████████████████████████████████████████| 30/30 [00:25<00:00,  1.19it/s]Custom logger is already specified. Specify more than one logger at same time is not thread safe.


In [8]:
scores

,parameter,rmse,.0,.1,.5,.9,1
0,BRAT,0.005758,0.000000,0.868485,1.000000,1.000000,1.954098
1,DEGT,2.333065,-184.170715,-27.160400,6.281616,30.211646,80.758423
2,DELFN,0.196182,-3.178211,9.006846,15.801858,26.496931,39.112564
3,DELN1,0.134876,-1.588493,3.747295,6.333039,10.703587,17.277214
4,DELVSV,0.059746,-0.800000,1.300000,2.300000,3.300000,18.200001
5,DPOIL,0.103597,-31.672470,-24.614487,-20.672855,-12.408699,2.366665
6,EGTC,1.825030,479.787018,594.069019,644.083496,760.351026,889.812988
7,EGTHDM,1.016720,-55.431641,19.796198,52.873406,73.615068,286.805603
8,EGTHDM_D,0.205973,-34.172806,-10.467921,0.000000,10.467924,34.172810
9,GEGTMC,1.120429,8.715698,52.337040,79.473602,141.741243,298.171021
